#### TODO:

function to clean all

* returns 2 cleaned corpora (2 diff versions of text prep)

function to train model on single corpus

* visualize 2 ways (with/without words)
* returns trained model

function to test a list of words against a specific word

* returns cos sims for each word relative to given word

* returns as list and df

etc/tbd

In [2]:
import praw
import pandas as pd
import re
import string
from collections import Counter
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from matplotlib import pyplot
%matplotlib inline

In [55]:
# iterate through reddit object, build dictionary, return dataframes
# return titles&posts df, plus whole df

def make_corpus(subreddit_object):
    '''
    takes subreddit api object,
    returns 2 dataframes:
    
    * entire subreddit, all columns
    * text corpus only
    
    '''
    
    columns_dict = {'title': [],
                   'url': [],
                   'date': [],
                   'score': [],
                   'n_comments': [],
                   'body': []}
    
    # build dictionary
    
    for post in subreddit_object:
        
        columns_dict['title'].append(post.title)
        
        columns_dict['url'].append(post.url)
        
        columns_dict['date'].append(post.created)
        
        columns_dict['score'].append(post.score)
        
        columns_dict['n_comments'].append(post.num_comments)
        
        columns_dict['body'].append(post.selftext)
    
    # convert to dataframe
    
    subreddit_df = pd.DataFrame(columns_dict)
    
    # split off text for df
    
    corpus_df = subreddit_df['title'] + subreddit_df['body']
    
    return subreddit_df, corpus_df
        

In [63]:
# text cleaning: 2 different versions
# functions for standardization

def light_clean(doc):
    '''
    lightly cleans input doc for
    word embedding training:
    
    * tokenizes
    * lowercase
    * removes punctuation & non-alpha characters
    
    returns clean document.
    
    '''
    
    # tokenize
    
    split_tokens = doc.split()
    
    # punctuation list
    
    punctuation = string.punctuation
    
    # set up regex filter
    
    rgx_punct_filter = re.compile('[%s]' % re.escape(punctuation))
    
    # apply filter
    
    no_punct_tokens = [rgx_punct_filter.sub('', char) for char in split_tokens]
    
    # alpha tokens only
    
    alpha_tokens = [t for t in no_punct_tokens if t.isalpha()]
    
    # lowercase
    
    lower_tokens = [t.lower() for t in alpha_tokens]
    
    return lower_tokens

def super_clean(doc):
    '''
    cleans a single input document.
    applies more processing than
    light_clean():
    
    * tokenizes
    * lowercase
    * removes punctuation & numbers
    * removes stopwords
    
    returns clean document.
    
    '''
    
    # tokenize
    
    split_tokens = doc.split()
    
    # grab punctuation list
    
    punctuation = string.punctuation
    
    # setup re filter
    
    rgx_punct_filter = re.compile('[%s]' % re.escape(punctuation))
    
    # apply filter
    
    no_punct_tokens = [rgx_punct_filter.sub('', char) for char in split_tokens]
    
    # take out numbers & any other non-alpha characters
    
    alpha_tokens = [i for i in no_punct_tokens if i.isalpha()]
    
    # lowercase
    
    lower_tokens = [t.lower() for t in alpha_tokens]
    
    # get stop words
    
    stop_words = set(stopwords.words('english'))
    
    # filter out stopwords
    
    go_tokens = [t for t in lower_tokens if not t in stop_words]
    
    # remove very short tokens
    
    clean_tokens = [word for word in go_tokens if len(word) > 1]
    
    return clean_tokens

In [64]:

def clean_all_docs(docs):
    '''
    takes corpus (list) as input
    iterates through input texts
    applies 2 different text cleaning functions
    appends cleaned documents to 2 seperate lists
    returns 2 lists of clean texts, and 2
    dataframes with vocabulary counts
    
    '''
    
    light_clean_docs = []
    
    light_clean_vocab = []
    
    super_clean_docs = []
    
    super_clean_vocab = []
    
    for doc in docs:
        
        # eliminate empty rows
        
        if doc:
        
            clean_doc_1 = light_clean(doc)
            
            clean_doc_2 = super_clean(doc)
        
            light_clean_docs.append(clean_doc_1)
            
            super_clean_docs.append(clean_doc_2)
            
    # get vocabulary
    
    for doc in light_clean_docs:
        
        for word in doc:
            
            light_clean_vocab.append(word)
            
    for doc in super_clean_docs:
        
        for word in doc:
            
            super_clean_vocab.append(word)
    
    # get word counts
    
    light_clean_counts = Counter(light_clean_vocab)
    
    super_clean_counts = Counter(super_clean_vocab)
    
    # get light clean text vocab to dataframe
    
    lc_vocab_df = pd.DataFrame.from_dict(light_clean_counts, orient='index')
    
    # sort by most frequent words
    
    lc_vocab_df = lc_vocab_df.sort_values(by=[0], ascending=False)
    
    # index
    
    light_clean_vocab = lc_vocab_df.reset_index()
    
    # get super clean text vocab
    
    sc_vocab_df = pd.DataFrame.from_dict(super_clean_counts, orient='index')
    
    sc_vocab_df = sc_vocab_df.sort_values(by=0, ascending=False)
    
    super_clean_vocab = sc_vocab_df.reset_index()
     
            
    return light_clean_docs, light_clean_vocab, super_clean_docs, super_clean_vocab


In [65]:
# test on a subreddit
# will use r/Feminism for this run

with open('r_feminism_script.txt') as f:
    
    feminism_script = f.read().strip()

with open('r_feminism_secret.txt') as f:
    
    feminism_secret = f.read().strip()

with open('reddit_id.txt') as f:
    
    pw = f.read().strip()
    
# get reddit object
    
reddit = praw.Reddit(client_id=feminism_script,
                    client_secret=feminism_secret,
                    user_agent='feminism',
                    username='neurodivergent_ai',
                    password=pw)

# get subreddit

feminism = reddit.subreddit('Feminism')

# get top posts

r_feminism = feminism.top(limit=1000)

for post in feminism.top(limit=5):
    
    print(post.title, '\n')

Some words from Maisie Williams 

This sadly happens all to often. 

This can never go away, this needs to continue to get posted over and over and over again. This should never be forgotten 

About abortion 

Trump Inauguration (top) vs. Women's March (bottom) 



In [66]:
r_feminism_df, r_feminism_corpus = make_corpus(r_feminism)

In [67]:

print('full dataframe .head(): \n')
print(r_feminism_df.head(), '\n')
print('text & title corpus .head(): \n')
print(r_feminism_corpus.head())

full dataframe .head(): 

                                               title  \
0                    Some words from Maisie Williams   
1                   This sadly happens all to often.   
2  This can never go away, this needs to continue...   
3                                     About abortion   
4  Trump Inauguration (top) vs. Women's March (bo...   

                                   url          date  score  n_comments body  
0   https://i.redd.it/ka0kgautv32z.jpg  1.496819e+09  10020         980       
1      https://i.imgur.com/0lLo035.jpg  1.509409e+09   5992         392       
2  https://i.redd.it/ar0tu4fi2jt01.jpg  1.524461e+09   3959         180       
3  https://i.redd.it/k4at3i36c6121.jpg  1.543484e+09   3234         213       
4       http://i.imgur.com/ivzyXdP.png  1.485059e+09   2727         166        

text & title corpus .head(): 

0                      Some words from Maisie Williams
1                     This sadly happens all to often.
2    This can never 

In [68]:
# return light_clean_docs, light_clean_vocab, super_clean_docs, super_clean_vocab

feminism_lc_docs, feminism_lc_vocab, feminism_sc_docs, feminism_sc_vocab = clean_all_docs(r_feminism_corpus)